# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902073


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:  10%|▉         | 3/31 [00:06<01:03,  2.26s/it]

Rendering models:  19%|█▉        | 6/31 [00:06<00:40,  1.60s/it]

Rendering models:  29%|██▉       | 9/31 [00:07<00:25,  1.14s/it]

Rendering models:  35%|███▌      | 11/31 [00:07<00:16,  1.22it/s]

...

Rendering models:  74%|███████▍  | 23/31 [00:08<00:01,  5.16it/s]

Rendering models:  84%|████████▍ | 26/31 [00:08<00:00,  6.65it/s]

Rendering models:  90%|█████████ | 28/31 [00:08<00:00,  7.39it/s]

Rendering models: 100%|██████████| 31/31 [00:08<00:00,  9.31it/s]

not-logged-in-bf4cef763ffa750a7579    2.246235
Thedunkmasta                          0.001781
not-logged-in-a9a92f25ca4015279a03    0.011694
EastonH                               0.003094
ElisabethB                            0.049061
not-logged-in-ea8ed61a7274cfb6a43b    0.006145
JEBala                                0.007806
SohaKawtharani                        0.003862
stekenxu                              0.043591
not-logged-in-fc6db6f8fca5399a40ba    0.001681
mattchat                              0.016590
not-logged-in-a0164d241f30411408f0    0.001159
not-logged-in-a0164d241f30411408f0    0.001500
not-logged-in-5f30b2bb9b7f9d49da41    0.067989
clairedeu                             0.000500
peakscience7                          0.125840
pangeli5                              0.000957
Saharisunshine                        0.002910
acapirala                             0.000620
not-logged-in-04ed9ae880f18762cd9e    0.102171
helios1235                            0.001166
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())